In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns


import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA


In [2]:
''' 
Define style specifying text details to save manual plot editing
General Ref: https://matplotlib.org/3.3.2/tutorials/introductory/customizing.html
'''
sns.set_theme(style = 'darkgrid', 
              rc = {'figure.titlesize' : 22, 
                    'figure.titleweight' : 'bold',
                    'axes.titleweight' : 'bold',
                    'axes.titlesize' : 22, 
                    'axes.labelweight' : 'bold',
                    'axes.labelsize' : 18,
                    'xtick.labelsize' : 14,
                    'ytick.labelsize' : 14
                   }
             )

# Data Import

In [6]:
train = pd.read_csv('./data/train.csv', index_col='id')
test = pd.read_csv('./data/test.csv', index_col='id')


train_poly = pd.read_csv('./data/train_exp.csv', index_col='id')
test_poly = pd.read_csv('./data/test_exp.csv', index_col='id')


X_train, X_test, y_train, y_test = train_test_split(train.drop('target', axis = 1),
                                                    train.loc[:, 'target'],
                                                    test_size = 0.5,
                                                    random_state = 42
                                                   )
X_train_poly, X_test_poly, y_train, y_test = train_test_split(train_poly,
                                                    train.loc[:, 'target'],
                                                    test_size = 0.5,
                                                    random_state = 42
                                                   )


# Naive Average Baseline

Due to the extremely weak relationship between provided features and the target outcome, a naive average should actually perform fairly well and serves as a reasonable first benchmark.

In [7]:
y_hat_mean = np.full(y_test.shape[0], np.mean(y_train))
rmse_mean = np.sqrt(mean_squared_error(y_hat_mean, y_test))

print('Pure Mean Test RMSE: {:.4f}'.format(rmse_mean))


Pure Mean Test RMSE: 0.7318


# OLS Baseline

None of the original features or derived features exhibited a strong linear relationship to the target. However, fitting a pure OLS model will measure any tangible gains collectively contributed by these features of a linear structure.

In [8]:
ols = LinearRegression()
ols.fit(X_train, y_train)

print('OLS Train R2: {:.4}'.format(ols.score(X_train, y_train)))
print('OLS Test R2: {:.4}'.format(ols.score(X_test, y_test)))

y_hat_ols = ols.predict(X_test)
rmse_ols = np.sqrt(mean_squared_error(y_hat_ols, y_test))

print('OLS Test RMSE: {:.4f}'.format(rmse_ols))
print('Improvement from pure mean: {:.2%}'.format((rmse_mean - rmse_ols)/rmse_mean))
print('\n')

ols_poly = LinearRegression()
ols_poly.fit(X_train_poly, y_train)

print('OLS with Poly Terms Train R2: {:.4}'.format(ols_poly.score(X_train_poly, y_train)))
print('OLS with Poly Terms Test R2: {:.4}'.format(ols_poly.score(X_test_poly, y_test)))

y_hat_ols_poly = ols_poly.predict(X_test_poly)
rmse_ols_poly = np.sqrt(mean_squared_error(y_hat_ols_poly, y_test))

print('OLS with Poly Terms Test RMSE: {:.4f}'.format(rmse_ols_poly))
print('Improvement from pure mean: {:.2%}'.format((rmse_mean - rmse_ols_poly)/rmse_mean))
print('Improvement from OLS without Poly Terms: {:.2%}'.format((rmse_ols - rmse_ols_poly)/rmse_ols))


OLS Train R2: 0.01895
OLS Test R2: 0.01827
OLS Test RMSE: 0.7251
Improvement from pure mean: 0.92%


OLS with Poly Terms Train R2: 0.04655
OLS with Poly Terms Test R2: 0.01049
OLS with Poly Terms Test RMSE: 0.7280
Improvement from pure mean: 0.53%
Improvement from OLS without Poly Terms: -0.40%


The linear OLS model improves a naive mean estimate by less than 1%. This further demonstrates the lack of linear relationship to the features and the target.

Incoporating the additional power transform variables and interaction terms improves model performance. This results primarily from the inclusion of interactive terms which displayed a stronger relationship to the target than the original features.

#### Detailed Model Summary

In [9]:
X = X_train_poly.copy()
X = sm.add_constant(X)
ols_model = sm.OLS(y_train, X)
results = ols_model.fit()

display(results.summary())

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     24.28
Date:                Thu, 28 Jan 2021   Prob (F-statistic):               0.00
Time:                        12:55:53   Log-Likelihood:            -1.6294e+05
No. Observations:              150000   AIC:                         3.265e+05
Df Residuals:                  149698   BIC:                         3.295e+05
Df Model:                         301                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 7.9058      0.002   4255.761      0.000       7.902       7.909
cont1                -0.0641      0.019     -3.299      0.001      -0.102      -0.026
cont2                -0.0178      0.010     -1.874      0.061      -0.036       0.001
cont3                -0.0129      0.008     -1.585      0.113      -0.029       0.003
cont4                 0.0841      0.010      8.448      0.000       0.065       0.104
cont5                -0.0119      0.009     -1.296      0.195      -0.030       0.006
cont6                -0.0855      0.026     -3.345      0.001      -0.136      -0.035
cont7                 0.0374      0.018      2.101      0.036       0.003       0.072
cont8                 0.0203      0.013      1.540      0.124      -0.006       0.046
cont9                -0.0482      0.022     -2.225      0.026      -0.091      -0.006
cont10                0.0249      0.021      1.176      0.240      -0.017       0.066
cont11                0.0334      0.022      1.541      0.123      -0.009       0.076
cont12               -0.0331      0.022     -1.474      0.141      -0.077       0.011
cont13               -0.0225      0.017     -1.321      0.186      -0.056       0.011
cont14                0.0760      0.009      8.121      0.000       0.058       0.094
cont1^2               0.1001      0.020      4.923      0.000       0.060       0.140
cont1_cont2           0.0163      0.014      1.153      0.249      -0.011       0.044
cont1_cont3          -0.0616      0.019     -3.307      0.001      -0.098      -0.025
cont1_cont4           0.2148      0.038      5.673      0.000       0.141       0.289
cont1_cont5           0.0162      0.040      0.409      0.683      -0.061       0.094
cont1_cont6          -0.0202      0.050     -0.403      0.687      -0.119       0.078
cont1_cont7           0.0443      0.030      1.457      0.145      -0.015       0.104
cont1_cont8           0.0105      0.046      0.228      0.820      -0.080       0.101
cont1_cont9           0.0224      0.024      0.938      0.348      -0.024       0.069
cont1_cont10         -0.0860      0.034     -2.563      0.010      -0.152      -0.020
cont1_cont11         -0.0261      0.042     -0.615      0.539      -0.109       0.057
cont1_cont12         -0.0778      0.046     -1.703      0.088      -0.167       0.012
cont1_cont13          0.1664      0.053      3.139      0.002       0.062       0.270
cont1_cont14          0.0300      0.039      0.774      0.439      -0.046       0.106
cont2^2              -0.1170      0.011    -11.072      0.000      -0.138      -0.096
cont2_cont3           0.0309      0.019      1.644      0.100      -0.006       0.068
cont2_cont4           0.0384      0.019      2.039      0.041       0.001       0.075
cont2_cont5          -0.0105      0.029     -0.359      0.720      -0.068

# Ridge Baseline

The heatmap of pairwise correlations in the exploratory analysis notebook revealed about half the features displayed reasonably strong pairwise relationships. This suggests multicollinearity could be an issue in fitting an OLS model. To mitigate error introduced by collinearity issues, an L2 norm can be utilized.

In [10]:
ridge_cv = RidgeCV(
    alphas = np.logspace(1, 4, 100),
    cv = 10
)
ridge_cv.fit(X_train, y_train)
y_hat_ridge = ridge_cv.predict(X_test)
rmse_ridge = np.sqrt(mean_squared_error(y_hat_ridge, y_test))

print('Ridge Train R2: {:.4}'.format(ridge_cv.score(X_train, y_train)))
print('Ridge Test R2: {:.4}'.format(ridge_cv.score(X_test, y_test)))
print('Optimal alpha: {}'.format(ridge_cv.alpha_))
print(' ')
print('Ridge Test RMSE: {:.4f}'.format(rmse_ridge))
print('Improvement from pure mean: {:.2%}'.format((rmse_mean - rmse_ridge)/rmse_mean))
print('Improvement from OLS: {:.2%}'.format((rmse_ols - rmse_ridge)/rmse_ols))

print('\n \n')


ridge_cv_poly = RidgeCV(
    alphas = np.logspace(1, 4, 100),
    cv = 10
)
ridge_cv_poly.fit(X_train_poly, y_train)
y_hat_ridge_poly = ridge_cv_poly.predict(X_test_poly)
rmse_ridge_poly = np.sqrt(mean_squared_error(y_hat_ridge_poly, y_test))

print('Ridge with Poly Terms Train R2: {:.4}'.format(ridge_cv.score(X_train, y_train)))
print('Ridge with Poly Terms Test R2: {:.4}'.format(ridge_cv.score(X_test, y_test)))
print('Optimal alpha: {}'.format(ridge_cv.alpha_))
print(' ')
print('Ridge with Poly Terms Test RMSE: {:.4f}'.format(rmse_ridge_poly))
print('Ridge with Poly Terms improvement from pure mean: {:.2%}'.format(
    (rmse_mean - rmse_ridge_poly)/rmse_mean))
print('Ridge with Poly Terms improvement from OLS with Poly Terms: {:.2%}'.format(
    (rmse_ols_poly - rmse_ridge_poly)/rmse_ols_poly))
print('Ridge with Poly Terms improvement from Ridge without Poly Terms: {:.2%}'.format(
    (rmse_ridge - rmse_ridge_poly)/rmse_ridge))



Ridge Train R2: 0.01895
Ridge Test R2: 0.01827
Optimal alpha: 11.497569953977356
 
Ridge Test RMSE: 0.7251
Improvement from pure mean: 0.92%
Improvement from OLS: -0.00%

 

Ridge with Poly Terms Train R2: 0.01895
Ridge with Poly Terms Test R2: 0.01827
Optimal alpha: 11.497569953977356
 
Ridge with Poly Terms Test RMSE: 0.7176
Ridge with Poly Terms improvement from pure mean: 1.94%
Ridge with Poly Terms improvement from OLS with Poly Terms: 1.42%
Ridge with Poly Terms improvement from Ridge without Poly Terms: 1.03%


The above results imply collinearity is not causing a drop in performance for the OLS model. Optimizing alpha through cross validation pushes alpha to towards the lowest value available. By requiring an alpha above 0, we force some divergence from the pure OLS model a find a reduction in performance on our internal test set.

# Lasso Baseline

An alternative approach to the collinearity issue is to eliminate correlated features and push towards a sparse feature set.

In [11]:
lasso_cv = LassoCV(
    alphas = np.logspace(-2, 2, 100),
    cv = 10
)
lasso_cv.fit(X_train, y_train)
y_hat_lasso = lasso_cv.predict(X_test)
rmse_lasso = np.sqrt(mean_squared_error(y_hat_lasso, y_test))

print('Lasso Train R2: {:.4}'.format(lasso_cv.score(X_train, y_train)))
print('Lasso Test R2: {:.4}'.format(lasso_cv.score(X_test, y_test)))
print('Optimal alpha: {}'.format(lasso_cv.alpha_))
print(' ')
print('Lasso Test RMSE: {:.4f}'.format(rmse_lasso))
print('Improvement from pure mean: {:.2%}'.format((rmse_mean - rmse_lasso)/rmse_mean))
print('Improvement from OLS: {:.2%}'.format((rmse_ols - rmse_lasso)/rmse_ols))

print('\n \n')

lasso_cv_poly = LassoCV(
    alphas = np.logspace(-2, 2, 100),
    cv = 10
)
lasso_cv_poly.fit(X_train_poly, y_train)
y_hat_lasso_poly = lasso_cv_poly.predict(X_test_poly)
rmse_lasso_poly = np.sqrt(mean_squared_error(y_hat_lasso_poly, y_test))

print('Lasso with Poly Terms Train R2: {:.4}'.format(lasso_cv.score(X_train, y_train)))
print('Lasso with Poly Terms Test R2: {:.4}'.format(lasso_cv.score(X_test, y_test)))
print('Optimal alpha: {}'.format(lasso_cv.alpha_))
print(' ')
print('Lasso with Poly Terms Test RMSE: {:.4f}'.format(rmse_lasso_poly))
print('Lasso with Poly Terms improvement from pure mean: {:.2%}'.format(
    (rmse_mean - rmse_lasso_poly)/rmse_mean))
print('Lasso with Poly Terms improvement from OLS with Poly Terms: {:.2%}'.format(
    (rmse_ols_poly - rmse_lasso_poly)/rmse_ols_poly))
print('Lasso with Poly Terms improvement from lasso without Poly Terms: {:.2%}'.format(
    (rmse_lasso - rmse_lasso_poly)/rmse_lasso))



Lasso Train R2: 0.0007672
Lasso Test R2: 0.0008007
Optimal alpha: 0.01
 
Lasso Test RMSE: 0.7315
Improvement from pure mean: 0.04%
Improvement from OLS: -0.89%

 

Lasso with Poly Terms Train R2: 0.0007672
Lasso with Poly Terms Test R2: 0.0008007
Optimal alpha: 0.01
 
Lasso with Poly Terms Test RMSE: 0.7234
Lasso with Poly Terms improvement from pure mean: 1.15%
Lasso with Poly Terms improvement from OLS with Poly Terms: 0.63%
Lasso with Poly Terms improvement from lasso without Poly Terms: 1.12%


Compared to utilizing an L2 norm, the Lasso L1 norm reduces the model performance by even more. This implies even those very weakly predictive features add some additional value not captured by correlated features.

# Principal Components

In [13]:
pca = PCA(random_state = 42)
pca.fit(X_train)
Z_train = pca.transform(X_train)
Z_test = pca.transform(X_test)


pcr = LinearRegression()
pcr.fit(Z_train, y_train)
y_hat_pcr = pcr.predict(Z_test)
rmse_pcr = np.sqrt(mean_squared_error(y_hat_pcr, y_test))

print('PCR Train R2: {:.4}'.format(pcr.score(Z_train, y_train)))
print('PCR Test R2: {:.4}'.format(pcr.score(Z_test, y_test)))
print('PCR Test RMSE: {:.4f}'.format(rmse_pcr))

print('Improvement from pure mean: {:.2%}'.format((rmse_mean - rmse_pcr)/rmse_mean))
print('Improvement from OLS: {:.2%}'.format((rmse_ols - rmse_pcr)/rmse_ols))



pca_poly = PCA(random_state = 42)
pca_poly.fit(X_train_poly)
Z_train_poly = pca_poly.transform(X_train_poly)
Z_test_poly = pca_poly.transform(X_test_poly)


pcr_poly = LinearRegression()
pcr_poly.fit(Z_train_poly, y_train)
y_hat_pcr_poly = pcr_poly.predict(Z_test_poly)
rmse_pcr_poly = np.sqrt(mean_squared_error(y_hat_pcr_poly, y_test))

print('PCR Train R2: {:.4}'.format(pcr_poly.score(Z_train_poly, y_train)))
print('PCR Test R2: {:.4}'.format(pcr_poly.score(Z_test_poly, y_test)))
print('PCR Test RMSE: {:.4f}'.format(rmse_pcr_poly))

print('Improvement from pure mean: {:.2%}'.format((rmse_mean - rmse_pcr_poly)/rmse_mean))
print('Improvement from OLS with Poly Features: {:.2%}'.format((rmse_ols_poly - rmse_pcr_poly)/rmse_ols_poly))
print('Improvement from Ridge with Poly Features: {:.2%}'.format((rmse_ridge_poly - rmse_pcr_poly)/rmse_ridge_poly))
print('Improvement from Lasso with Poly Features: {:.2%}'.format((rmse_lasso_poly - rmse_pcr_poly)/rmse_lasso_poly))



PCR Train R2: 0.01895
PCR Test R2: 0.01827
PCR Test RMSE: 0.7251
Improvement from pure mean: 0.92%
Improvement from OLS: 0.00%
PCR Train R2: 0.04655
PCR Test R2: 0.0105
PCR Test RMSE: 0.7280
Improvement from pure mean: 0.53%
Improvement from OLS with Poly Features: 0.00%
Improvement from Ridge with Poly Features: -1.44%
Improvement from Lasso with Poly Features: -0.64%


In [14]:
(pca.explained_variance_ratio_)[:20]

array([0.41143793, 0.13220389, 0.10149224, 0.06825866, 0.06213106,
       0.05422367, 0.04171451, 0.03159362, 0.02616827, 0.0184101 ,
       0.01598846, 0.013077  , 0.01243212, 0.01086848])

# Benchmarks Summary

In [15]:
rmse_summary = {'basic' : [rmse_mean, rmse_ols, rmse_ridge, rmse_lasso, rmse_pcr], 
                'poly' : [rmse_mean, rmse_ols_poly, rmse_ridge_poly, rmse_lasso_poly, rmse_pcr_poly]}
rmse_summary = pd.DataFrame(rmse_summary, index = ['mean', 'ols', 'ridge', 'lasso', 'pcr'])
round(rmse_summary, 4)

,basic,poly
mean,0.7318,0.7318
ols,0.7251,0.7280
ridge,0.7251,0.7176
lasso,0.7315,0.7234
pcr,0.7251,0.7280
